In [ ]:
import pandas as pd

df = pd.read_csv('/Users/keshavsaraogi/Desktop/indorama/eureka-data/raw-csv/sales_packaging.csv')

In [ ]:
# df.head(15)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
missing_percent = df.isnull().mean() * 100
missing_percent = missing_percent.sort_values(ascending=False)

print(missing_percent.round(2))

In [ ]:
import pandas as pd

nan_counts = df.isnull().sum()
nan_counts = nan_counts[nan_counts > 0]

hash_counts = (
    df.select_dtypes(include='object')
    .apply(lambda col: col.astype(str).str.count('#').sum())
)
hash_counts = hash_counts[hash_counts > 0]

combined = pd.DataFrame({
    "NaN_Count": nan_counts,
    "#_Count": hash_counts
}).fillna(0).astype(int)

combined["Total_Issues"] = combined["NaN_Count"] + combined["#_Count"]
combined = combined.sort_values("Total_Issues", ascending=False)

print("📊 Columns with NaN or '#' values and their counts:")
print(combined)


In [ ]:
for col in df.columns:
    print(f"📌 Column: {col}")
    print(f"🔢 Unique Values ({df[col].nunique(dropna=False)}):")
    print(df[col].unique())
    print("-" * 80)

In [ ]:
df.columns.to_list()

## Handling Missing Values

In [ ]:
df.columns.to_list()

In [ ]:
df.rename(columns={
    "Company Code": "Company ID",
    "Sales Organization": "Sales Organization ID",
    "Plant": "Plant ID",
    "Material": "Material ID",
    "Incoterms 1": "Incoterms Type",
    
}, inplace=True)

In [ ]:
# Convert to datetime WITHOUT custom format
df['Sales Invoice Date'] = pd.to_datetime(df['Sales Invoice Date'], errors='coerce')

# (Optional) Format back to string as 'YYYY-MM-DD' (if needed for export or SQL)
df['Sales Invoice Date'] = df['Sales Invoice Date'].dt.strftime('%Y-%m-%d')


In [ ]:
#output_path = "/Users/keshavsaraogi/Desktop/indorama/eureka-data/clean-csv/cleaned_sales_packaging.csv"
#df.to_csv(output_path, index=False, encoding="utf-8-sig")

#print(f"✅ DataFrame saved to: {output_path}")

In [ ]:
# Convert 'Sales Invoice Date' to datetime if not already
df['Sales Invoice Date'] = pd.to_datetime(df['Sales Invoice Date'], errors='coerce')

# Filter for the specific customer and year
df_filtered = df[
    (df['Customer'] == 'SEVEN UP BOTTLING CO PLC') & 
    (df['Sales Invoice Date'].dt.year == 2024)
]

# Create a 'month' column in YYYY-MM format
df_filtered['month'] = df_filtered['Sales Invoice Date'].dt.to_period('M').astype(str)

# Calculate profit and group by month
result = (
    df_filtered
    .groupby('month')
    .apply(lambda x: x['Invoice Net value'].sum() - x['Tax amount'].sum())
    .reset_index(name='profit')
    .sort_values('month')
)

print(result)